In [2]:
!pip install earthengine-api
!pip install rasterio
!pip install google-colab
!pip install pyarrow

In [117]:
import ee
import json

ee.Authenticate()
ee.Initialize(project='mse-uhi')

Extracting Image Data (Singapore)

In [3]:
import ee
ee.Initialize()

# Define the region of interest
singapore = ee.Geometry.Rectangle([103.59, 1.13, 104.07, 1.47])

# Load the Landsat 8 Collection 2 Tier 2 Image Collection
Landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T2_L2')

# Filter by date and location for the year 2022
filtered_collection = Landsat8_collection.filterDate('2022-01-01', '2022-12-31').filterBounds(singapore)

def export_each(image):
    # Format date for file naming
    date = image.date().format('YYYYMMdd').getInfo()
    
    # Select bands, assuming we need all available bands
    image_selected = image.toFloat()  # Convert to float if needed for processing

    # Export each image to Drive
    task = ee.batch.Export.image.toDrive(
        image=image_selected,
        description=f'L8_{date}',
        folder='L8_Data',
        scale=30,
        region=singapore.getInfo()['coordinates'],
        fileFormat='GeoTIFF'
    )
    task.start()
    print(f"Task {task.id} started for {date}. Checking status...")

    # Check and print the status of the task
    while task.active():
        task_status = task.status()
        print(f"Task {task.id} for {date}: {task_status['state']}")
        if task_status['state'] == 'COMPLETED':
            print(f"Task {task.id} for {date} completed successfully.")
            break
        elif task_status['state'] == 'FAILED':
            print(f"Task {task.id} for {date} failed with error: {task_status['error_message']}")
            break

# Apply the function to each image in the collection
image_list = filtered_collection.toList(filtered_collection.size())
for i in range(image_list.size().getInfo()):
    image = ee.Image(image_list.get(i))
    export_each(image)

Task FAYMYFP6THWOOVNRIXQUXWXY started for 20220620. Checking status...
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: READY
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: READY
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: READY
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING
Task FAYMYFP6THWOOVNRIXQUXWXY for 20220620: RUNNING

In [29]:
import rasterio
import pandas as pd
import numpy as np
import os

year = "2022"

data_directory = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\{year}"

# Loop through each file in the directory
for filename in os.listdir(data_directory):
    prefix = f"L8_{year}"
    if filename.startswith(prefix):
        file_path = os.path.join(data_directory, filename)

        # Open the GeoTIFF file
        with rasterio.open(file_path) as src:
            array = src.read()
            nodata_values = src.nodatavals
            band_names = src.descriptions

            # Reshape the array to a 2D array where each row is a pixel
            reshaped_array = array.transpose(1, 2, 0).reshape(-1, len(band_names))
            bands_df = pd.DataFrame(reshaped_array, columns=band_names)
            bands_df['x'] = np.tile(np.arange(src.width), src.height)
            bands_df['y'] = np.repeat(np.arange(src.height), src.width)
            bands_df['B10_degC'] = bands_df['ST_B10'] * 0.00341802 + 149 - 273.15
            
            # Replace NoData values appropriately
            for i, nodata_value in enumerate(nodata_values):
                if nodata_value is not None:
                    bands_df.iloc[:, i].replace(nodata_value, np.nan, inplace=True)

            # Save the bands data to CSV
            bands_csv_path = os.path.join(data_directory, f'Landsat8_Bands_2022{filename[-8:-4]}.csv')
            bands_df.to_csv(bands_csv_path, index=False)

            print(f"Saved band data to {bands_csv_path}") 

            # Extract and save metadata to CSV
            metadata = src.meta
            metadata_df = pd.DataFrame([metadata])
            metadata_csv_path = os.path.join(data_directory, f'Landsat8_Metadata_2022{filename[-8:-4]}.csv')
            metadata_df.to_csv(metadata_csv_path, index=False)

            print(f"Saved metadata to {metadata_csv_path}") 

Saved band data to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\2022\Landsat8_Bands_20220620.csv
Saved metadata to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\2022\Landsat8_Metadata_20220620.csv
Saved band data to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\2022\Landsat8_Bands_20220908.csv
Saved metadata to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\2022\Landsat8_Metadata_20220908.csv
Saved band data to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\2022\Landsat8_Bands_20221010.csv
Saved metadata to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\2022\Landsat8_Metadata_20221010.csv
Saved band data to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\2022\Landsat8_Bands_20221026.csv
Saved metadata to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\2022\Landsat8_Metadata_20221026.csv
Saved band data to C:\LocalOneDrive\Docu